In [ ]:
!pip install PyMuPDF

!pip install -U spacy pdfminer.six

# STEP 1: Mount Google Drive
from google.colab import drive
import os, json, random
import spacy
from spacy.training.example import Example
import re


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.7 MB/s eta 0:00:00


In [ ]:
import numpy as nlp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
dataset_path = '/content/drive/MyDrive/ResumesJsonAnnotated'


In [ ]:
# STEP 2: Preprocess Dataset
def preprocess_kaggle_dataset(folder_path):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            full_path = os.path.join(folder_path, filename)
            try:
                with open(full_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)



                    # ✅ USE THIS INSTEAD:
                    text = data.get("text", "").encode("utf-8", "ignore").decode("utf-8", "ignore")

                    annotations = data.get("annotations", [])
                    valid_entities = []

                    for ann in annotations:
                        if len(ann) != 3:
                            continue
                        start, end, label = ann
                        if not isinstance(start, int) or not isinstance(end, int):
                            continue
                        if start >= end or end > len(text):
                            continue
                        if ":" in label:
                            label = label.split(":")[0].strip()
                        entity_text = text[start:end].strip()
                        if not entity_text:
                            continue
                        new_start = text.find(entity_text, start)
                        new_end = new_start + len(entity_text)
                        if new_start >= 0 and new_end <= len(text):
                            valid_entities.append((new_start, new_end, label))

                    # Remove overlapping spans
                    valid_entities = sorted(valid_entities, key=lambda x: x[0])
                    non_overlap = []
                    last_end = -1
                    for start, end, label in valid_entities:
                        if start >= last_end:
                            non_overlap.append((start, end, label))
                            last_end = end

                    all_data.append((text, {"entities": non_overlap}))
            except Exception as e:
                print(f"❌ Failed to load {filename}: {e}")
    return all_data


In [ ]:

# Split the dataset into training and test sets
dataset_path = '/content/drive/MyDrive/ResumesJsonAnnotated'
TRAIN_DATA = preprocess_kaggle_dataset(dataset_path)
print("\n✅ Loaded samples:", len(TRAIN_DATA))

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(TRAIN_DATA, test_size=0.2, random_state=42)

print(f"Training examples: {len(train_data)}")
print(f"Testing examples: {len(test_data)}")





✅ Loaded samples: 5029
Training examples: 4023
Testing examples: 1006


In [ ]:
# STEP 3: Train spaCy NER model
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

for _, annotations in train_data:
    for start, end, label in annotations["entities"]:
        ner.add_label(label)

nlp.begin_training()
for epoch in range(10):
    random.shuffle(train_data)
    losses = {}
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses)
    print(f"📘 Epoch {epoch+1} - Loss: {losses}")

Streaming output truncated to the last 5000 lines.
G-8,F/F,ABUL FAZAL 
ENCLAVE-1,AZAD 
RO..." with entities "[(90, 95, 'SKILL'), (150, 155, 'SKILL'), (156, 159...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Juan Antonio Aguilar Garib
Curriculum Vitae
Fecha:..." with entities "[(27, 43, 'SKILL'), (173, 175, 'SKILL'), (522, 530...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "RESUME
B.PRASATH
E.Mail: niceprasath888@gmail.com
..." with entities "[(0, 6, 'SK

📘 Epoch 1 - Loss: {'ner': np.float32(326498.78)}
📘 Epoch 2 - Loss: {'ner': np.float32(275437.75)}
📘 Epoch 3 - Loss: {'ner': np.float32(257788.08)}
📘 Epoch 4 - Loss: {'ner': np.float32(243227.95)}
📘 Epoch 5 - Loss: {'ner': np.float32(232107.83)}
📘 Epoch 6 - Loss: {'ner': np.float32(223000.89)}
📘 Epoch 7 - Loss: {'ner': np.float32(215491.67)}
📘 Epoch 8 - Loss: {'ner': np.float32(208754.38)}
📘 Epoch 9 - Loss: {'ner': np.float32(202487.98)}
📘 Epoch 10 - Loss: {'ner': np.float32(196148.47)}


In [ ]:
from spacy.scorer import Scorer
from spacy.training import Example

def evaluate_model(nlp, test_data):
    examples = []
    for text, annotations in test_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(nlp(text), annotations)
        examples.append(example)

    # Compute scores
    scorer = Scorer()
    scores = scorer.score(examples)

    print("📊 Evaluation Metrics:")
    print(f"Precision: {scores['ents_p']:.2f}")
    print(f"Recall: {scores['ents_r']:.2f}")
    print(f"F1-score: {scores['ents_f']:.2f}")

# Evaluate after model training
evaluate_model(nlp, test_data)


/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "AYUSH JAIN Contact: 9759734784 Email:  ayushjain92..." with entities "[(31, 36, 'SKILL'), (53, 58, 'SKILL'), (59, 62, 'S...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "MILTON JOHN D’SOUZA
Mother Palace, Wagholi (Souzal..." with entities "[(68, 69, 'SKILL'), (127, 128, 'SKILL'), (171, 176...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

📊 Evaluation Metrics:
Precision: 0.68
Recall: 0.83
F1-score: 0.75


In [ ]:
# Metric	What it means
# Precision	Of all the predicted entities, how many were correct?
# Recall	Of all the actual (true) entities, how many were found?
# F1-Score	Harmonic mean of Precision and Recall — overall quality

In [ ]:
# STEP 5: Define REGEX extractors

def extract_email(text):
    match = re.search(r'[\w\.-]+@[\w\.-]+\.\w+', text)
    return match.group() if match else None

def extract_phone(text):
    # Clean out common separators
    text = text.replace(" ", "").replace(".", "").replace("-", "")

    # Match common formats including international +91 and 10-digit numbers
    match = re.search(r'(\+91)?\d{10}\b', text)
    if match:
        return match.group()

    # Fallback: Match formats like 123-456-7890 or (123) 456-7890
    match = re.search(r'\b(?:\(?\d{3}\)?[\s\-]?)\d{3}[\s\-]?\d{4}\b', text)
    if match:
        return match.group().strip()

    return None


def extract_linkedin(text):
    cleaned = text.replace(" ", "").replace("linkedin.com", "https://linkedin.com")
    match = re.search(r'https://linkedin\.com/[^\s]+', cleaned)
    return match.group() if match else None

def extract_name(text):
    lines = text.strip().split("\n")
    for line in lines[:5]:
        if re.match(r"^[A-Z][a-zA-Z]+ [A-Z][a-zA-Z]+", line.strip()):
            return line.strip()
    return None




In [53]:


def extract_year(text):
    lines = text.splitlines()
    found_education = False
    all_years = []

    for i, line in enumerate(lines):
        if re.search(r'\beducation\b', line, re.IGNORECASE):
            found_education = True
            continue
        if found_education:
            # Stop if we hit another section
            if re.search(r'\b(projects|experience|skills|summary|certifications|achievements|objective)\b', line, re.IGNORECASE):
                break
            # Extract all 4-digit years from lines under education
            years = re.findall(r'\b(19[7-9]\d|20[0-2]\d)\b', line)
            all_years.extend([int(y) for y in years])

    if all_years:
        return [str(max(all_years))]  # Return the highest year as string
    return []


import re

def extract_college(text):
    lines = text.splitlines()
    # Education section keywords
    education_keywords = r'\b(education|academic|qualifications|degrees|studies)\b'
    # Section breakers (expanded to catch "achievement" explicitly)
    section_breakers = r'\b(projects|experience|skills|summary|achievement|achievements|certifications|objective|extra\s*curricular|awards)\b'
    # Robust college name pattern
    college_pattern = r"(?:[A-Z][a-zA-Z0-9.\-&']*(?:\s+[A-Z][a-zA-Z0-9.\-&']*)*)\s*(?:University|College|Institute|Academy|Polytechnic|School of\s+[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*|(?:[A-Z]{2,4}\s*){1,2})\b(?:\s*,\s*[A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*)*)?(?:\s*\([^\)]+\))?"

    inside_education = False
    possible_colleges = []

    for line in lines:
        line = line.strip()
        if not line:
            continue  # Skip empty lines

        # Detect education section
        if re.search(education_keywords, line, re.IGNORECASE):
            inside_education = True
            continue

        # Stop at next major section
        if inside_education and re.search(section_breakers, line, re.IGNORECASE):
            inside_education = False
            break

        # Process lines in education section
        if inside_education:
            # Remove leading dashes, bullets, or pipes
            line = re.sub(r'^-|\s*•|\s*\|+', '', line).strip()
            # Look for college names
            match = re.search(college_pattern, line, re.IGNORECASE)
            if match:
                college_name = match.group(0).strip()
                # Clean up trailing CGPA or other parenthetical content
                college_name = re.sub(r'\s*\([^\)]+\)$', '', college_name).strip()
                # Filter out non-college names (e.g., high schools, section headers)
                if not re.search(r'\b(high\s+school|secondary\s+school|prep\s+school|public\s+school|achievement|extra\s*curricular)\b', college_name, re.IGNORECASE):
                    possible_colleges.append(college_name)

    # Prioritize and return results
    if possible_colleges:
        # Prioritize names with "University", "Institute", "Academy", or "College"
        for college in possible_colleges:
            if re.search(r'\b(university|college|institute|academy|polytechnic)\b', college, re.IGNORECASE):
                return [college]
        # Fallback to first valid match
        return [possible_colleges[0]]

    return []


import re

def extract_degree(text):
    # Try to isolate the Education section (case-insensitive)
    education_block = re.search(r'education\s*(.*?)(?=\n[A-Z][a-zA-Z\s]+:|\Z)', text, re.IGNORECASE | re.DOTALL)
    if not education_block:
        return []

    edu_text = education_block.group(1)

    # Use regex to capture only degrees mentioned — case-insensitive, no hardcoding unrelated terms
    match = re.findall(r'\b(?:B\.?\s?Tech|M\.?\s?Tech|B\.?\s?E|M\.?\s?E|B\.?\s?Sc|M\.?\s?Sc|Ph\.?\s?D|MBA)\b', edu_text, flags=re.IGNORECASE)

    # Normalize and deduplicate
    return list(set(m.strip().replace(" ", "").replace(".", ".") for m in match))


def extract_designation(text):
    titles = re.findall(r'(?i)\b(engineer|developer|data scientist|intern|analyst|consultant|scientist)\b', text)
    return list(set(titles))


def extract_companies(text):
    lines = text.splitlines()
    in_experience = False

    # Common job title words (to exclude from company matching)
    # job_titles = {
    #     "engineer", "developer", "scientist", "analyst", "intern", "consultant",
    #     "manager", "designer", "lead", "specialist", "tester", "officer", "assistant"
    # }

    # Read job titles from CSV (first column only)
    job_df = pd.read_csv('/content/sample_data/job_titles.csv')  # adjust path if needed
    job_titles = set(job_df.iloc[:, 0].str.lower().str.strip())

    # Company name regex: 1–3 capitalized words, optional suffixes
    company_pattern = re.compile(
        r'\b([A-Z][a-zA-Z]*(?:\s+[A-Z][a-zA-Z]*){0,2}'  # 1 to 3 words
        r'(?:\s+(Technologies|Solutions|Systems|Group|Partners|Associates|Consulting))?'  # optional suffix
        r'(?:\s+(Inc\.?|LLC|Ltd\.?|Corp\.?|Co\.?|GmbH))?)\b'
    )

    for i, line in enumerate(lines):
        if re.search(r'\b(work\s*experience|experience|employment|professional\s+experience)\b', line, re.IGNORECASE):
            in_experience = True
            # Look at next 4 lines max
            for j in range(i + 1, min(i + 5, len(lines))):
                candidate = lines[j].strip()

                # Ignore lines that clearly contain job titles
                if any(word.lower() in job_titles for word in candidate.lower().split()):
                    continue

                match = company_pattern.search(candidate)
                if match:
                    name = match.group(1).strip()
                    if 1 <= len(name.split()) <= 3:
                        return [name]
            break

    return []

In [64]:
import pandas as pd
skill_df = pd.read_csv('/content/sample_data/skills.csv')
TECH_SKILLS = set(skill_df.iloc[:, 0].str.lower().str.strip())

def filter_skills(skills):
    return list(set([s for s in skills if s.lower() in TECH_SKILLS]))

def parse_resume(text, nlp_model):
    doc = nlp_model(text)
    import pandas as pd


    ner_results = {
        "Skills": [], "Degree": [], "Designation": [], "College": [],
        "Companies": [], "Graduation Year": []
    }

    for ent in doc.ents:
        label = ent.label_.lower()
        value = ent.text.strip()
        if "skill" in label:
            ner_results["Skills"].append(value)
        elif "degree" in label:
            ner_results["Degree"].append(value)
        elif "designation" in label:
            ner_results["Designation"].append(value)
        elif "college" in label or "institute" in label:
            ner_results["College"].append(value)
        elif "company" in label:
            ner_results["Companies"].append(value)
        elif "year" in label:
            ner_results["Graduation Year"].append(value)

    final_result = {
        "Name": extract_name(text),
        "Email": extract_email(text),
        "Phone": extract_phone(text),
        "LinkedIn": extract_linkedin(text),
        **ner_results
    }

    # Fallbacks using custom extractors
    if not final_result["Graduation Year"]:
        final_result["Graduation Year"] = extract_year(text)
    if not final_result["College"]:
        final_result["College"] = extract_college(text)
    if not final_result["Degree"]:
        final_result["Degree"] = extract_degree(text)
    if not final_result["Designation"]:
        final_result["Designation"] = extract_designation(text)
    if not final_result["Companies"]:
        final_result["Companies"] = extract_companies(text)

    # ✅ Only filter tech skills if the resume indicates engineering
    is_engineer = bool(re.search(r'\b(B\.?Tech|M\.?Tech|Engineer|Engineering|B\.?E\.?|Bachelor of Technology|Master of Technology)\b', text, re.IGNORECASE))

    if is_engineer:
        final_result["Skills"] = filter_skills(final_result["Skills"])
    else:
        final_result["Skills"] = list(set(final_result["Skills"]))  # keep all, unfiltered
    # 🔍 Fallback regex: look for bullets or known keywords like "Skills", "Frameworks", "Languages", "Tools"
    skill_patterns = re.findall(
        r'(?:skills|frameworks|technologies|tools|languages|libraries|software)\s*[:\-]?\s*(.*?)(?=\n\s*(?:projects|education|experience|certifications|activities|summary|profile|$))',
        text,
        re.IGNORECASE | re.DOTALL
    )

    # Also capture lines that begin with bullet points •
    bullet_skills = re.findall(r'•\s*(.+)', text)

    combined_skill_text = ' '.join(skill_patterns + bullet_skills)
    combined_skill_text = re.sub(r'[\n]+', ' ', combined_skill_text)
    combined_skill_text = re.sub(r'[^\w\s\+\#\.\-\,]', '', combined_skill_text)  # Keep useful symbols

    # Split by common delimiters
    candidate_skills = re.split(r'[,•;:\-\t]+', combined_skill_text)
    additional_skills = set()

    for skill in candidate_skills:
        skill = skill.strip().lower()

        # ❌ Filter out numeric entries or things that look like years (e.g. 2019, 2023)
        if skill.isdigit() or re.match(r'\b(19|20)\d{2}\b', skill):
            continue
        if any(char.isdigit() for char in skill):
            # allow known tech like html5, 3d only if it's short and no more than 2 digits
            if not re.match(r'^(html5|css3|3d|2d|nodejs|vue3|react18)$', skill):
                continue

        if 1 < len(skill) <= 30 and len(skill.split()) <= 4:
            additional_skills.add(skill)

    # Merge all extracted skills
    existing_skills = set(s.strip().lower() for s in final_result.get("Skills", []))
    combined_skills = existing_skills.union(additional_skills)

    # Normalization and smart splitting
    clean_skills = set()

    for skill in combined_skills:
        skill = re.sub(r'\s+', ' ', skill.strip().lower())  # normalize whitespace
        parts = re.split(r',|\band\b|/|\+|-|_', skill)

        for part in parts:
            subparts = part.split()
            for sp in subparts:
                sp = sp.strip()

                # ❌ Skip entries that are just numbers or too short/long
                if sp.isdigit() or len(sp) <= 1 or len(sp) > 30:
                    continue

                if re.match(r'\b(19|20)\d{2}\b', sp):  # Skip years
                    continue

                # Final filter for numeric-only or mostly numeric
                if sum(c.isdigit() for c in sp) > 2:
                    continue

                clean_skills.add(sp)

    # Final clean sorted skill list
    final_result["Skills"] = sorted(clean_skills)

    return final_result


In [43]:
import spacy
import joblib
import os

# Save trained spaCy NER model
output_dir = "/content/resume_ner_model"
nlp.to_disk(output_dir)
print("✅ spaCy NER model saved to:", output_dir)

# Save helper objects (example: skills_list, patterns)
skills_list = ['Python', 'NLP', 'Data Science']  # Replace with your actual list
joblib.dump(skills_list, "skills_list.pkl")
print("✅ Skills list saved.")


✅ spaCy NER model saved to: /content/resume_ner_model
✅ Skills list saved.


In [44]:
from google.colab import files
files.download("skills_list.pkl")
!zip -r resume_ner_model.zip resume_ner_model
files.download("resume_ner_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

updating: resume_ner_model/ (stored 0%)
updating: resume_ner_model/ner/ (stored 0%)
updating: resume_ner_model/ner/moves (deflated 40%)
updating: resume_ner_model/ner/cfg (deflated 33%)
updating: resume_ner_model/ner/model (deflated 7%)
updating: resume_ner_model/config.cfg (deflated 59%)
updating: resume_ner_model/vocab/ (stored 0%)
updating: resume_ner_model/vocab/vectors.cfg (stored 0%)
updating: resume_ner_model/vocab/vectors (deflated 45%)
updating: resume_ner_model/vocab/lookups.bin (stored 0%)
updating: resume_ner_model/vocab/strings.json (deflated 75%)
updating: resume_ner_model/vocab/key2row (stored 0%)
updating: resume_ner_model/meta.json (deflated 49%)
updating: resume_ner_model/tokenizer (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import spacy
import joblib

# Upload your model folder and helper files manually using the upload button or mount Drive
from google.colab import files
uploaded = files.upload()  # Upload `skills_list.pkl` and `resume_ner_model.zip`

# Unzip and load spaCy model
!unzip -o resume_ner_model.zip
nlp = spacy.load("/content/resume_ner_model")
print("✅ spaCy NER model loaded.")

# Load skills list
skills_list = joblib.load("skills_list.pkl")
print("✅ Skills list loaded.")


Saving resume_ner_model.zip to resume_ner_model (1).zip
Saving skills_list.pkl to skills_list (1).pkl
Archive:  resume_ner_model.zip
  inflating: resume_ner_model/ner/moves  
  inflating: resume_ner_model/ner/cfg  
  inflating: resume_ner_model/ner/model  
  inflating: resume_ner_model/config.cfg  
 extracting: resume_ner_model/vocab/vectors.cfg  
  inflating: resume_ner_model/vocab/vectors  
 extracting: resume_ner_model/vocab/lookups.bin  
  inflating: resume_ner_model/vocab/strings.json  
 extracting: resume_ner_model/vocab/key2row  
  inflating: resume_ner_model/meta.json  
  inflating: resume_ner_model/tokenizer  
✅ spaCy NER model loaded.
✅ Skills list loaded.


In [65]:
# STEP 7: Use on new resume text (PDF text or test string)
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

# text = extract_text_from_pdf('/content/sample_data/Sample_Resume (1).pdf')  # or any string input
# result = parse_resume(text, nlp)
# from pprint import pprint
# pprint(result)

In [66]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# === File uploader ===
upload = widgets.FileUpload(accept='.pdf', multiple=False, layout=widgets.Layout(width='200px'))

# === Output display fields ===
name_box = widgets.Text(description='👤 Name:', layout=widgets.Layout(width='100%'))
email_box = widgets.Text(description='📧 Email:', layout=widgets.Layout(width='100%'))
phone_box = widgets.Text(description='📱 Phone:', layout=widgets.Layout(width='100%'))
linkedin_box = widgets.Text(description='🔗 LinkedIn:', layout=widgets.Layout(width='100%'))

college_box = widgets.Text(description='🎓 College:', layout=widgets.Layout(width='100%'))
company_box = widgets.Text(description='🏢 Company:', layout=widgets.Layout(width='100%'))
degree_box = widgets.Text(description='📘 Degree:', layout=widgets.Layout(width='100%'))
designation_box = widgets.Text(description='💼 Role:', layout=widgets.Layout(width='100%'))
year_box = widgets.Text(description='📅 Grad Year:', layout=widgets.Layout(width='100%'))

skills_box = widgets.Textarea(description='🛠 Skills:', layout=widgets.Layout(width='100%', height='80px'))

# === Parse button ===
parse_button = widgets.Button(description='📄 Extract Info', button_style='success')

# === Output area ===
output_area = widgets.Output()

# === Handler ===
def on_parse_clicked(b):
    with output_area:
        clear_output()
        if not upload.value:
            print("❗ Please upload a PDF first.")
            return

        # Save uploaded file
        for name, file_info in upload.value.items():
            with open(name, 'wb') as f:
                f.write(file_info['content'])

        # Parse resume
        text = extract_text_from_pdf(name)
        result = parse_resume(text, nlp)

        # Fill output fields
        name_box.value = result.get("Name") or ""
        email_box.value = result.get("Email") or ""
        phone_box.value = result.get("Phone") or ""
        linkedin_box.value = result.get("LinkedIn") or ""

        college_box.value = ', '.join(result.get("College", []))
        company_box.value = ', '.join(result.get("Companies", []))
        degree_box.value = ', '.join(result.get("Degree", []))
        designation_box.value = ', '.join(result.get("Designation", []))
        year_box.value = ', '.join(result.get("Graduation Year", []))
        skills_box.value = ', '.join(result.get("Skills", []))

        print("✅ Resume processed successfully!")

# === Connect button click ===
parse_button.on_click(on_parse_clicked)

# === Layout ===
form = widgets.VBox([
    widgets.HTML("<h3 style='color: #2b6cb0;'>📤 Upload Resume (PDF)</h3>"),
    upload,
    parse_button,
    widgets.HTML("<hr><h4 style='color:#2f855a;'>📋 Extracted Information:</h4>"),
    name_box, email_box, phone_box, linkedin_box,
    college_box, company_box, degree_box, designation_box, year_box,
    skills_box,
    output_area
])

display(form)


In [67]:
import os
import fitz  # PyMuPDF
import pandas as pd

def parse_multiple_resumes(resume_folder, nlp):
    results = []
    for file in os.listdir(resume_folder):
        if file.endswith(".pdf"):
            with fitz.open(os.path.join(resume_folder, file)) as doc:
                text = ""
                for page in doc:
                    text += page.get_text()

            parsed = parse_resume(text, nlp)  # your existing parsing logic
            results.append(parsed)

    df = pd.DataFrame(results)
    df.to_excel("parsed_resumes.xlsx", index=False)
    print("✅ All resumes parsed and saved to parsed_resumes.xlsx")


In [68]:
parse_multiple_resumes("/content/sample_data/Untitled Folder",nlp)

✅ All resumes parsed and saved to parsed_resumes.xlsx
